In [47]:
import pandas as pd
import pymssql

In [57]:
conn = pymssql.connect('10.1.0.38:1433', 'automacao_gs', '2711!@#GS', 'KMFAG')
# Cria o cursor.
cursor = conn.cursor()
# Realiza a consulta da tabela da pedidos.
cursor.execute("""
                SELECT
                    EMP.Nome,
                    PED.EST_Codigo,
                    EST.Nome AS EST_Nome,
                    FRN.NOME AS FORNECEDOR,
                    EST.CNPJCPF AS CNPJ,
                    CASE
                        WHEN PED.STATUS = 2 THEN 'REJEITADO'
                        WHEN PED.STATUS = 1 THEN 'APROVADO'
                        WHEN PED.STATUS = 0 THEN 'AGUARDANDO APROVACAO'
                        ELSE 'OUTRO' 
                    END AS STATUS,
                    PED.[DATA] AS DT_PEDIDO,
                    CASE
                        WHEN VCP.[Data] IS NOT NULL
                            THEN VCP.[Data]
                        ELSE VCP_ENT.[Data]
                    END AS DT_VENCIMENTO,
                    DPE.[Path] AS CAMINHO_ARQUIVO,
                    PED.CondicaoPagamento AS CONDICAO_PAGAMENTO,
                    CPG.DOCUMENTO AS DOC,
                    CPG.ValorBruto AS VALOR_BRUTO,
                    PED.CODIGO AS CODIGO_PEDIDO,
                    CONCAT(CPG.Codigo, ENT.CPG_CODIGO) AS CODIGO_CONTAS_PAGAR,
                    CONCAT( VCP.Sequencial, VCP_ENT.Sequencial ) AS SEQUENCIAL_CONTAS_PAGAR,
                    PED.CampoExtra3 AS CONTAS_AVULSAS,
                    PED.CampoExtra2 AS FORMA_PAGAMENTO,
                    PED.CampoExtra1 AS NOTA,
                    PED.CampoExtra4 AS SETOR,
                    PED.CAMPOEXTRA6 AS TIPO,
                    CASE
                        WHEN ENT.CODIGO IS NOT NULL
                            THEN 'PEDIDO COM NOTA'
                        ELSE 'PEDIDO AVULSO'
                    END AS TIPO_PEDIDO    
                FROM PED	
                LEFT JOIN DPE 				 ON DPE.EMP_Codigo = PED.EMP_CODIGO 				AND DPE.PED_Codigo = PED.CODIGO
                LEFT JOIN CPG 				 ON CPG.EMP_Codigo = PED.EMP_CODIGO 				AND CPG.Codigo = PED.CPG_CODIGO
                LEFT JOIN VCP 				 ON VCP.EMP_Codigo = CPG.EMP_Codigo 				AND VCP.CPG_Codigo = CPG.Codigo
                LEFT JOIN EST 				 ON EST.EMP_Codigo = PED.EMP_Codigo 				AND EST.Codigo = PED.EST_Codigo
                LEFT JOIN EMP 				 ON EST.EMP_Codigo = EMP.Codigo
                LEFT JOIN ENT 				 ON ENT.EMP_Codigo = PED.EMP_CODIGO         AND ENT.PED_CODIGO = PED.CODIGO
                LEFT JOIN CPG CPG_ENT  ON CPG_ENT.EMP_CODIGO = ENT.EMP_CODIGO     AND CPG_ENT.CODIGO = ENT.CPG_CODIGO
                LEFT JOIN VCP VCP_ENT  ON VCP_ENT.EMP_Codigo = CPG_ENT.EMP_Codigo AND VCP_ENT.CPG_Codigo = CPG_ENT.Codigo
                LEFT JOIN FRN          ON PED.EMP_Codigo = FRN.EMP_Codigo			    AND PED.FRN_Codigo = FRN.Codigo;
                """)

In [58]:
columns = ['Grupo', 'EST_Codigo', 'EST_nome', 'fornecedor', 'cnpj', 'status','dt_pedido', 'dt_vencimento', 'path', 'cond_pag', 'num_nota', 'valor', 'cod_ped', 'COD_cpg', 'sequencial', 'cont_avuls', 'form_pag', 'nota', 'setor', 'tipo', 'tipo_pedido']
df = pd.DataFrame(cursor.fetchall(), columns=columns)

In [59]:
qtdd_registros = len(df)
qtdd_valores = df[df['valor'].isna()]
qtdd_valores = qtdd_registros - len(qtdd_valores)

In [60]:
print(qtdd_registros)
print(qtdd_valores)

38610
10213


In [61]:
df

,Grupo,EST_Codigo,EST_nome,fornecedor,cnpj,status,dt_pedido,dt_vencimento,path,cond_pag,...,valor,cod_ped,COD_cpg,sequencial,cont_avuls,form_pag,nota,setor,tipo,tipo_pedido
0,CONDONAL,0007,CONDO SERVICOS DE APOIO ADMINISTRATIVO LTDA (P&R),WEVERSON SOUZA DOS ANJOS,26101538000186,APROVADO,2025-02-17,2025-02-24,Y:\CELULA FISCAL - FORTES\CONDONAL\GESTÃO DE C...,24/02/2025,...,900.00,20250200020,20250200963,1,SIM,BOLETO,NÃO,GESTÃO DE CONDOMÍNIO,PAGAMENTO,PEDIDO AVULSO
1,CONDONAL,0010,CONDONAL SERVICOS E ADMINISTRACAO LTDA,THIAGO PÁDUA DE SOUZA,00712911000141,APROVADO,2025-02-18,2025-02-24,Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...,VENCIMENTO: 24/02/2025 - CHAVE PIX: 119.924.30...,...,150.00,20250200023,20250200948,1,SIM,PIX,NÃO,OPERACIONAL,PAGAMENTO,PEDIDO AVULSO
2,CONDONAL,0016,VITORIAGATTI SEGURANCA E VIGILANCIA LTDA,PHENIX ASSESSORIA E CONSULTORIA EMPRESAR,12094349000119,APROVADO,2025-02-18,2025-02-28,Y:\CELULA FISCAL - FORTES\CONDONAL\DP\PHENIX -...,28/02/2025,...,129.50,20250200029,20250201096,1,SIM,BOLETO,NAO,DP,PAGAMENTO,PEDIDO AVULSO
3,CONDONAL,0016,VITORIAGATTI SEGURANCA E VIGILANCIA LTDA,PHENIX ASSESSORIA E CONSULTORIA EMPRESAR,12094349000119,APROVADO,2025-02-18,2025-02-28,Y:\CELULA FISCAL - FORTES\CONDONAL\DP\PHENIX -...,28/02/2025,...,129.50,20250200029,20250201096,1,SIM,BOLETO,NAO,DP,PAGAMENTO,PEDIDO AVULSO
4,CONDONAL,0003,CONDO SERVICOS EM CONDOMINIOS LTDA,CONDO SERVICOS EM CONDOMINIOS LTDA,21549002000104,APROVADO,2025-02-18,2025-02-24,Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...,PAGAR EM 24/02/2025,...,3021.38,20250200033,20250200955,1,SIM,TRANSFERÊNCIA,NÃO,OPERACIONAL,PAGAMENTO,PEDIDO AVULSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38605,FOLK,0002,FOLK PORTARIA REMOTA LTDA,SIMPLES SOLUTIONS COME DE EQUIP ELETRONICOS LTDA,23253380000171,AGUARDANDO APROVACAO,2025-07-09,NaT,Y:\CELULA FISCAL - FORTES\FOLK\MARCOSS\PREMIUM...,28/07/25; 25/08/25; 22/09/25; 20/10/25; 17/11/...,...,NaN,20250700122,,,NÃO,BOLETO,SIM,COMPRAS,PAGAMENTO,PEDIDO AVULSO
38606,FOLK,0002,FOLK PORTARIA REMOTA LTDA,SIMPLES SOLUTIONS COME DE EQUIP ELETRONICOS LTDA,23253380000171,AGUARDANDO APROVACAO,2025-07-09,NaT,Y:\CELULA FISCAL - FORTES\FOLK\MARCOSS\PREMIUM...,28/07/25; 25/08/25; 22/09/25; 20/10/25; 17/11/...,...,NaN,20250700122,,,NÃO,BOLETO,SIM,COMPRAS,PAGAMENTO,PEDIDO AVULSO
38607,FOLK,0002,FOLK PORTARIA REMOTA LTDA,SIMPLES SOLUTIONS COME DE EQUIP ELETRONICOS LTDA,23253380000171,AGUARDANDO APROVACAO,2025-07-09,NaT,Y:\CELULA FISCAL - FORTES\FOLK\MARCOSS\PREMIUM...,28/07/25; 25/08/25; 22/09/25; 20/10/25; 17/11/...,...,NaN,20250700122,,,NÃO,BOLETO,SIM,COMPRAS,PAGAMENTO,PEDIDO AVULSO
38608,FOLK,0001,FK PR LOCACAO DE EQUIPAMENTOS LTDA,SIMPLES SOLUTIONS COMERCIO DE EQUIP ELETRONICO...,35803103000174,APROVADO,2024-11-08,NaT,Y:\CELULA FISCAL - FORTES\FOLK\LEANDROV\NF-311...,25/11/2024; 25/12/2024; 25/01/2025; 25/02/2025...,...,NaN,20250800001,,,NÃO,BOLETO,SIM,COMPRAS,PAGAMENTO,PEDIDO COM NOTA


In [ ]:
data = pd.to_datetime(df.iloc[0]['dt_vencimento'], format="%d/%m/%Y")
print()

21

In [55]:
df.iloc[0]['dt_vencimento'].strftime('%d-%m-%Y')

'21-02-2025'

In [33]:
pd.isna(df.iloc[38510]['valor'])

True

In [34]:
pd.isna(df.iloc[0]['valor'])

False

In [35]:
pd.isna(df.iloc[1]['num_nota'])

True

In [36]:
pd.isna(df.iloc[0]['num_nota'])

False

In [45]:
teste = " hitec R$ 1785 EDIFICIO residencial "
teste.upper()

' HITEC R$ 1785 EDIFICIO RESIDENCIAL '